In [3]:
import pandas as pd
import numpy as np 
import yfinance as yf
import datetime

start = '2010-01-01'
end = '2023-03-01'

gold_stock = yf.download('GC=F', start=start, end=end, interval='1d', auto_adjust=True)
gold_stock.to_csv('gold_stock_data.csv')

[*********************100%***********************]  1 of 1 completed


In [9]:
gold_stock = pd.read_csv("transformed_data.csv")
gold_stock.drop(columns=gold_stock.columns[0], axis=1, inplace=True)
gold_stock

,Date,Target,Target-3,Target-2,Target-1
0,2010-01-07,1133.099976,1117.699951,1118.099976,1135.900024
1,2010-01-08,1138.199951,1118.099976,1135.900024,1133.099976
2,2010-01-11,1150.699951,1135.900024,1133.099976,1138.199951
3,2010-01-12,1128.900024,1133.099976,1138.199951,1150.699951
4,2010-01-13,1136.400024,1138.199951,1150.699951,1128.900024
...,...,...,...,...,...
3302,2023-02-22,1832.000000,1842.000000,1840.400024,1833.000000
3303,2023-02-23,1818.000000,1840.400024,1833.000000,1832.000000
3304,2023-02-24,1808.800049,1833.000000,1832.000000,1818.000000
3305,2023-02-27,1817.000000,1832.000000,1818.000000,1808.800049


In [10]:
def window_split_columns(df):
    #convert to numpy array
    df = df.to_numpy()

    dates = df[:,0]

    Y = df[:,1]

    X = df[:, 2:]
    X = X.reshape((len(dates), X.shape[1], 1))

    return dates, X.astype(np.float32), Y.astype(np.float32)

dates, X, Y = window_split_columns(gold_stock)



In [12]:
percent80 = int(len(dates)*.8)
percent90 = int(len(dates)*.9)

dates_train, X_train, y_train = dates[:percent80], X[:percent80], Y[:percent80]
dates_val, X_val, y_val = dates[percent80:percent90], X[percent80:percent90], Y[percent80:percent90]
dates_test, X_test, y_test = dates[percent90:], X[percent90:], Y[percent90:]


In [17]:
import tensorflow as tf
from tensorflow import keras
from keras import transformer_model


# Define the model architecture
def transformer_model(input_dim, output_dim, num_heads, ff_dim, dropout_rate, max_len):
    inputs = keras.layers.Input(shape=(max_len, input_dim))
    x = keras.layers.BatchNormalization()(inputs)
    x = keras.layers.Dense(ff_dim, activation="relu")(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    x = keras.layers.Dense(output_dim, activation="linear")(x)
    x = keras.layers.Transformer()
    model = keras.Model(inputs=inputs, outputs=x)
    return model

# Set hyperparameters
input_dim = 1 # Number of input features
output_dim = 1 # Number of output features
num_heads = 2 # Number of attention heads
ff_dim = 32 # Dimensionality of the feedforward network
dropout_rate = 0.2 # Dropout rate
max_len = 3 # Maximum length of input sequence

# Initialize the model
model = transformer_model(input_dim, output_dim, num_heads, ff_dim, dropout_rate, max_len)

# Compile the model
model.compile(loss="mse", optimizer="adam")

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=100, validation_data=(X_test, y_test))

# Evaluate the model
model.evaluate(X_test, y_test)



Epoch 1/100
27/27 [==============================] - 2s 12ms/step - loss: 1893735.2500 - val_loss: 3274475.5000
Epoch 2/100
27/27 [==============================] - 0s 4ms/step - loss: 1892981.5000 - val_loss: 3269586.2500
Epoch 3/100
27/27 [==============================] - 0s 4ms/step - loss: 1892207.3750 - val_loss: 3266257.2500
Epoch 4/100
27/27 [==============================] - 0s 4ms/step - loss: 1891354.7500 - val_loss: 3263406.7500
Epoch 5/100
27/27 [==============================] - 0s 4ms/step - loss: 1890389.0000 - val_loss: 3260359.5000
Epoch 6/100
27/27 [==============================] - 0s 4ms/step - loss: 1889237.0000 - val_loss: 3256803.5000
Epoch 7/100
27/27 [==============================] - 0s 4ms/step - loss: 1887915.7500 - val_loss: 3252418.2500
Epoch 8/100
27/27 [==============================] - 0s 4ms/step - loss: 1886335.3750 - val_loss: 3247199.2500
Epoch 9/100
27/27 [==============================] - 0s 4ms/step - loss: 1884517.0000 - val_loss: 3240926.0000


43477.9921875